In [4]:
from pathlib import Path
import csv
try:
    folder_path = Path("GetData/Rust/get_data_rust/Data/Simulations")
    reader = csv.DictReader(open(folder_path / "parameters.csv"))
    par_dict = next(reader) 
except:
    raise ValueError("Invalid folder provided")
sim_num = int(par_dict['Simulatiton Number'])

In [19]:
import tensorflow as tf
import numpy as np

class RBM:
    def __init__(self, num_visible, num_hidden, learning_rate=0.1):
        self.num_visible = num_visible
        self.num_hidden = num_hidden
        self.learning_rate = learning_rate
        
        # Initialize weights and biases
        self.W = tf.Variable(tf.random.normal([num_visible, num_hidden], stddev=0.01))
        self.visible_bias = tf.Variable(tf.zeros([num_visible]))
        self.hidden_bias = tf.Variable(tf.zeros([num_hidden]))

    def sample_h_given_v(self, v):
        p_h_given_v = tf.nn.sigmoid(tf.matmul(v, self.W) + self.hidden_bias)
        return tf.nn.relu(tf.sign(p_h_given_v - tf.random.uniform(tf.shape(p_h_given_v))))

    def sample_v_given_h(self, h):
        p_v_given_h = tf.nn.sigmoid(tf.matmul(h, tf.transpose(self.W)) + self.visible_bias)
        return tf.nn.relu(tf.sign(p_v_given_h - tf.random.uniform(tf.shape(p_v_given_h))))

    def train(self, input_data, num_epochs=10, batch_size=10):
        num_samples = input_data.shape[0]
        for epoch in range(num_epochs):
            for i in range(0, num_samples - batch_size + 1, batch_size):
                batch = input_data[i:i+batch_size]
                positive_hidden_activations = self.sample_h_given_v(batch)
                positive_associations = tf.matmul(tf.transpose(batch), positive_hidden_activations)

                negative_visible_activations = self.sample_v_given_h(positive_hidden_activations)
                negative_hidden_activations = self.sample_h_given_v(negative_visible_activations)
                negative_associations = tf.matmul(tf.transpose(negative_visible_activations), negative_hidden_activations)

                self.W.assign_add(self.learning_rate * (positive_associations - negative_associations) / batch_size)
                self.visible_bias.assign_add(self.learning_rate * tf.reduce_mean(batch - negative_visible_activations, axis=0))
                self.hidden_bias.assign_add(self.learning_rate * tf.reduce_mean(positive_hidden_activations - negative_hidden_activations, axis=0))

            error = tf.reduce_mean(tf.square(batch - negative_visible_activations))
            print(f"Epoch {epoch+1}/{num_epochs}, Reconstruction Error: {error.numpy()}")

    def generate(self, num_samples):
        # Gibbs sampling to generate new samples
        samples = tf.random.uniform(shape=[num_samples, self.num_visible])
        for _ in range(1000):  # Perform 1000 Gibbs sampling steps for mixing
            hidden_samples = self.sample_h_given_v(samples)
            samples = self.sample_v_given_h(hidden_samples)
        return samples

# Example usage:
if __name__ == "__main__":
    # Create a sample dataset
    data = tf.convert_to_tensor([[1, 1, 1, 0, 0, 0],
                     [1, 0, 1, 0, 0, 0],
                     [1, 1, 1, 0, 0, 0],
                     [0, 0, 1, 1, 1, 0],
                     [0, 0, 1, 1, 0, 0]], dtype=tf.float32)

    rbm = RBM(num_visible=data.shape[1], num_hidden=2, learning_rate=0.1)
    rbm.train(data, num_epochs=100, batch_size=1)

    # Generate samples from the trained RBM
    generated_samples = rbm.generate(5)
    print("Generated samples:")
    print(generated_samples.numpy())


Epoch 1/100, Reconstruction Error: 0.5
Epoch 2/100, Reconstruction Error: 0.8333333134651184
Epoch 3/100, Reconstruction Error: 0.5
Epoch 4/100, Reconstruction Error: 0.3333333432674408
Epoch 5/100, Reconstruction Error: 0.3333333432674408
Epoch 6/100, Reconstruction Error: 0.0
Epoch 7/100, Reconstruction Error: 0.3333333432674408
Epoch 8/100, Reconstruction Error: 0.3333333432674408
Epoch 9/100, Reconstruction Error: 0.3333333432674408
Epoch 10/100, Reconstruction Error: 0.0
Epoch 11/100, Reconstruction Error: 0.5
Epoch 12/100, Reconstruction Error: 0.3333333432674408
Epoch 13/100, Reconstruction Error: 0.1666666716337204
Epoch 14/100, Reconstruction Error: 0.0
Epoch 15/100, Reconstruction Error: 0.3333333432674408
Epoch 16/100, Reconstruction Error: 0.5
Epoch 17/100, Reconstruction Error: 0.1666666716337204
Epoch 18/100, Reconstruction Error: 0.3333333432674408
Epoch 19/100, Reconstruction Error: 0.5
Epoch 20/100, Reconstruction Error: 0.5
Epoch 21/100, Reconstruction Error: 0.333333